<h1>2b. Machine Learning using tf.estimator </h1>

In this notebook, we will create a machine learning model using tf.estimator and evaluate its performance.  The dataset is rather small (7700 samples), so we can do it all in-memory.  We will also simply pass the raw data in as-is. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import shutil

print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


Read data created in the previous chapter.

In [2]:
# In CSV, label is the first column, after the features, followed by the key
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS) - 1]
LABEL = CSV_COLUMNS[0]

df_train = pd.read_csv('./taxi-train.csv', header = None, names = CSV_COLUMNS)
df_valid = pd.read_csv('./taxi-valid.csv', header = None, names = CSV_COLUMNS)
df_test = pd.read_csv('./taxi-test.csv', header = None, names = CSV_COLUMNS)

<h2> Train and eval input functions to read from Pandas Dataframe </h2>

In [3]:
def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
  )

In [4]:
def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

Our input function for predictions is the same except we don't provide a label

In [5]:
def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

### Create feature columns for estimator

In [6]:
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

<h3> Linear Regression with tf.Estimator framework </h3>

In [7]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = make_feature_cols(), model_dir = OUTDIR)

model.train(input_fn = make_train_input_fn(df_train, num_epochs = 10))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_save_checkpoints_secs': 600, '_task_id': 0, '_model_dir': 'taxi_trained', '_evaluation_master': '', '_tf_random_seed': None, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_session_config': None, '_is_chief': True, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7d83db8d30>}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 28971.373, st

Evaluate on the validation data (we should defer using the test data to after we have selected a final model).

In [8]:
def print_rmse(model, df):
  metrics = model.evaluate(input_fn = make_eval_input_fn(df))
  print('RMSE on dataset = {}'.format(np.sqrt(metrics['average_loss'])))
print_rmse(model, df_valid)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-25-20:13:14
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-25-20:13:14
INFO:tensorflow:Saving dict for global step 608: average_loss = 108.95201, global_step = 608, loss = 12957.507
RMSE on dataset = 10.438008308410645


This is nowhere near our benchmark (RMSE of $6 or so on this data), but it serves to demonstrate what TensorFlow code looks like.  Let's use this model for prediction.

In [9]:
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
for items in predictions:
  print(items)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-608
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predictions': array([11.270502], dtype=float32)}
{'predictions': array([11.267868], dtype=float32)}
{'predictions': array([11.26876], dtype=float32)}
{'predictions': array([11.266226], dtype=float32)}
{'predictions': array([11.270233], dtype=float32)}
{'predictions': array([11.270004], dtype=float32)}
{'predictions': array([11.268537], dtype=float32)}
{'predictions': array([11.26857], dtype=float32)}
{'predictions': array([11.270393], dtype=float32)}
{'predictions': array([11.268168], dtype=float32)}
{'predictions': array([11.270502], dtype=float32)}
{'predictions': array([11.270675], dtype=float32)}
{'predictions': array([11.263723], dtype=float32)}
{'predictions': array([11.267775], dtype=float32)}
{'predictions': array([11

{'predictions': array([11.271089], dtype=float32)}
{'predictions': array([11.268187], dtype=float32)}
{'predictions': array([11.269949], dtype=float32)}
{'predictions': array([11.271057], dtype=float32)}
{'predictions': array([11.382124], dtype=float32)}
{'predictions': array([11.268636], dtype=float32)}
{'predictions': array([11.268797], dtype=float32)}
{'predictions': array([11.324217], dtype=float32)}
{'predictions': array([11.370139], dtype=float32)}
{'predictions': array([11.270648], dtype=float32)}
{'predictions': array([11.545792], dtype=float32)}
{'predictions': array([11.268607], dtype=float32)}
{'predictions': array([11.269366], dtype=float32)}
{'predictions': array([11.27036], dtype=float32)}
{'predictions': array([11.266356], dtype=float32)}
{'predictions': array([11.262831], dtype=float32)}
{'predictions': array([11.548704], dtype=float32)}
{'predictions': array([11.267746], dtype=float32)}
{'predictions': array([11.268154], dtype=float32)}
{'predictions': array([11.270212

{'predictions': array([11.268648], dtype=float32)}
{'predictions': array([11.267632], dtype=float32)}
{'predictions': array([11.37951], dtype=float32)}
{'predictions': array([11.268898], dtype=float32)}
{'predictions': array([11.26861], dtype=float32)}
{'predictions': array([11.491344], dtype=float32)}
{'predictions': array([11.267384], dtype=float32)}
{'predictions': array([11.323852], dtype=float32)}
{'predictions': array([11.323717], dtype=float32)}
{'predictions': array([11.270116], dtype=float32)}
{'predictions': array([11.271118], dtype=float32)}
{'predictions': array([11.491422], dtype=float32)}
{'predictions': array([11.26803], dtype=float32)}
{'predictions': array([11.2644615], dtype=float32)}
{'predictions': array([11.489744], dtype=float32)}
{'predictions': array([11.490782], dtype=float32)}
{'predictions': array([11.2680855], dtype=float32)}
{'predictions': array([11.32339], dtype=float32)}
{'predictions': array([11.265438], dtype=float32)}
{'predictions': array([11.268559]

{'predictions': array([11.267655], dtype=float32)}
{'predictions': array([11.269018], dtype=float32)}
{'predictions': array([11.268574], dtype=float32)}
{'predictions': array([11.269244], dtype=float32)}
{'predictions': array([11.268721], dtype=float32)}
{'predictions': array([11.268491], dtype=float32)}
{'predictions': array([11.268473], dtype=float32)}
{'predictions': array([11.378374], dtype=float32)}
{'predictions': array([11.271284], dtype=float32)}
{'predictions': array([11.267813], dtype=float32)}
{'predictions': array([11.324054], dtype=float32)}
{'predictions': array([11.432868], dtype=float32)}
{'predictions': array([11.261982], dtype=float32)}
{'predictions': array([11.271091], dtype=float32)}
{'predictions': array([11.270295], dtype=float32)}
{'predictions': array([11.489866], dtype=float32)}
{'predictions': array([11.269748], dtype=float32)}
{'predictions': array([11.324498], dtype=float32)}
{'predictions': array([11.270285], dtype=float32)}
{'predictions': array([11.32501

{'predictions': array([11.268452], dtype=float32)}
{'predictions': array([11.263919], dtype=float32)}
{'predictions': array([11.267224], dtype=float32)}
{'predictions': array([11.324829], dtype=float32)}
{'predictions': array([11.322967], dtype=float32)}
{'predictions': array([11.489267], dtype=float32)}
{'predictions': array([11.268042], dtype=float32)}
{'predictions': array([11.268385], dtype=float32)}
{'predictions': array([11.269093], dtype=float32)}
{'predictions': array([11.434174], dtype=float32)}
{'predictions': array([11.323696], dtype=float32)}
{'predictions': array([11.271108], dtype=float32)}
{'predictions': array([11.319727], dtype=float32)}
{'predictions': array([11.26854], dtype=float32)}
{'predictions': array([11.268972], dtype=float32)}
{'predictions': array([11.487006], dtype=float32)}
{'predictions': array([11.490059], dtype=float32)}
{'predictions': array([11.268619], dtype=float32)}
{'predictions': array([11.268091], dtype=float32)}
{'predictions': array([11.546814

{'predictions': array([11.32564], dtype=float32)}
{'predictions': array([11.270621], dtype=float32)}
{'predictions': array([11.267797], dtype=float32)}
{'predictions': array([11.364498], dtype=float32)}
{'predictions': array([11.268135], dtype=float32)}
{'predictions': array([11.253614], dtype=float32)}
{'predictions': array([11.269077], dtype=float32)}
{'predictions': array([11.2683735], dtype=float32)}
{'predictions': array([11.323135], dtype=float32)}
{'predictions': array([11.270859], dtype=float32)}
{'predictions': array([11.490307], dtype=float32)}
{'predictions': array([11.268045], dtype=float32)}
{'predictions': array([11.267268], dtype=float32)}
{'predictions': array([11.37834], dtype=float32)}
{'predictions': array([11.268658], dtype=float32)}
{'predictions': array([11.324762], dtype=float32)}
{'predictions': array([11.269617], dtype=float32)}
{'predictions': array([11.263941], dtype=float32)}
{'predictions': array([11.2688265], dtype=float32)}
{'predictions': array([11.26983

This explains why the RMSE was so high -- the model essentially predicts the same amount for every trip.  Would a more complex model help? Let's try using a deep neural network.  The code to do this is quite straightforward as well.

<h3> Deep Neural Network regression </h3>

In [10]:
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNRegressor(hidden_units = [32, 8, 2],
      feature_columns = make_feature_cols(), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 100));
print_rmse(model, df_valid)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_save_checkpoints_secs': 600, '_task_id': 0, '_model_dir': 'taxi_trained', '_evaluation_master': '', '_tf_random_seed': None, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_train_distribute': None, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_save_checkpoints_steps': None, '_session_config': None, '_is_chief': True, '_log_step_count_steps': 100, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7d6413a710>}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into taxi_trained/model.ckpt.
INFO:tensorflow:loss = 15198.797, st

We are not beating our benchmark with either model ... what's up?  Well, we may be using TensorFlow for Machine Learning, but we are not yet using it well.  That's what the rest of this course is about!

But, for the record, let's say we had to choose between the two models. We'd choose the one with the lower validation error. Finally, we'd measure the RMSE on the test data with this chosen model.

<h2> Benchmark dataset </h2>

Let's do this on the benchmark dataset.

In [11]:
import google.datalab.bigquery as bq
import numpy as np
import pandas as pd

def create_query(phase, EVERY_N):
  """
  phase: 1 = train 2 = valid
  """
  base_query = """
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  EXTRACT(DAYOFWEEK FROM pickup_datetime) * 1.0 AS dayofweek,
  EXTRACT(HOUR FROM pickup_datetime) * 1.0 AS hourofday,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  CONCAT(CAST(pickup_datetime AS STRING), CAST(pickup_longitude AS STRING), CAST(pickup_latitude AS STRING), CAST(dropoff_latitude AS STRING), CAST(dropoff_longitude AS STRING)) AS key
FROM
  `nyc-tlc.yellow.trips`
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  """

  if EVERY_N == None:
    if phase < 2:
      # Training
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) < 2".format(base_query)
    else:
      # Validation
      query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 4) = {1}".format(base_query, phase)
  else:
    query = "{0} AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), {1}) = {2}".format(base_query, EVERY_N, phase)
    
  return query

query = create_query(2, 100000)
df = bq.Query(query).execute().result().to_dataframe()

In [12]:
print_rmse(model, df)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-25-20:17:11
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from taxi_trained/model.ckpt-6071
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-25-20:17:12
INFO:tensorflow:Saving dict for global step 6071: average_loss = 88.9361, global_step = 6071, loss = 11300.055
RMSE on dataset = 9.430593490600586


RMSE on benchmark dataset is <b>9.41</b> (your results will vary because of random seeds).

This is not only way more than our original benchmark of 6.00, but it doesn't even beat our distance-based rule's RMSE of 8.02.

Fear not -- you have learned how to write a TensorFlow model, but not to do all the things that you will have to do to your ML model performant. We will do this in the next chapters. In this chapter though, we will get our TensorFlow model ready for these improvements.

In a software sense, the rest of the labs in this chapter will be about refactoring the code so that we can improve it.

## Challenge Exercise

Create a neural network that is capable of finding the volume of a cylinder given the radius of its base (r) and its height (h). Assume that the radius and height of the cylinder are both in the range 0.5 to 2.0. Simulate the necessary training dataset.
<p>
Hint (highlight to see):
<p style='color:white'>
The input features will be r and h and the label will be $\pi r^2 h$
Create random values for r and h and compute V.
Your dataset will consist of r, h and V.
Then, use a DNN regressor.
Make sure to generate enough data.
</p>

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License